<a href="https://colab.research.google.com/github/ykim71/georgia_analysis/blob/main/(1)_Georgia_data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# need access to the folder(s) below:

#%cd /content/drive/MyDrive/Georgia/


## Data load/prep for translation

### previous data (for tracking history purpose-only)

In [ ]:
# not needed anymore
#%cd /content/drive/MyDrive/Marquee data

import pandas as pd
import os
path = os.getcwd()
files = os.listdir(path)

files_xls = [f for f in files if f[-4:] == 'xlsx']
files_xls


In [ ]:
df_list = []

for f in files_xls:
    data = pd.read_excel(f, dtype=object)
    df_list.append(data)

In [ ]:
df_frame = pd.concat(df_list, axis=0, ignore_index=True)


In [ ]:
df_frame_nna = df_frame.dropna(subset=['article_text'])

In [ ]:
df_frame_nna = df_frame_nna.sort_values(by='source')

In [ ]:
samples_count = df_frame_nna.groupby('source').count()

In [ ]:
min(df_frame_nna.date_published)

In [ ]:
max(df_frame_nna.date_published)

In [ ]:
# remove line breaks
df_frame_nna['article_text'] = df_frame_nna['article_text'].astype(str)
df_frame_nna['article_text'] = df_frame_nna['article_text'].map(lambda x: x.replace('\n', ' '))

In [ ]:
%cd /content/drive/MyDrive/Georgia

df_frame_nna.to_pickle('Georgia_data_all.pkl')

In [ ]:
import pandas as pd

df_frame_nna = pd.read_pickle('Georgia_data_all.pkl')

In [ ]:
import pandas as pd

df_frame_nna = pd.read_pickle('Georgia_data_all_ner_lem.pkl')

In [ ]:
count = df_frame_nna.groupby('source')['article_text'].count()

In [ ]:
count.to_excel('Georgia_source_list.xlsx')

### updated version

In [ ]:
import pandas as pd

df = pd.read_excel('/content/drive/MyDrive/Georgia (1)/Web Scraping /Updated Marquee data/delivery_2023-03-02.xlsx')

In [ ]:
df.columns

In [ ]:
temp = df.groupby(['source']).count().reset_index()
temp[temp['source'].str.contains("Northside Neighbor|Paulding Neighbor|South Fulton Neighbor|Marietta Daily Journal")]
# Northside Neighbor, Paulding Neighbor, South Fulton Neighbor, Marietta Daily Journal

In [ ]:
df_nna = df.dropna(subset=['article_text']) # this was for checking to see how many duplicated articles (some sources function like AP publishing the same articles)

In [ ]:
print(len(df_nna))
print(len(df))

In [ ]:
df_nna_sample = df_nna.groupby('source').apply(lambda x: x.sample(10, replace=False)).reset_index(drop=True) # see random 10 samplesl but there are some sources that have less than 10 articles published
df_nna_sample.to_excel('df_nna_sample.xlsx')

In [ ]:
len(df_nna_sample)

### translation

In [ ]:
%cd /content/drive/MyDrive/Georgia_YK

In [ ]:
# this file is from df_nna_sample.xlsx with checking what language is used in the article on Google Sheet 
# how other languages than english is written in the random 10 articles 
df_sample_lang = pd.read_excel('df_nna_sample_wlang.xlsx') 

In [ ]:
df_sample_lang = df_sample_lang[['source', 'LANGUAGE']].dropna(subset=['LANGUAGE']).drop_duplicates()

In [ ]:
df_sample_lang_article = pd.merge(df_sample_lang, df_nna, how="left", on="source")

In [ ]:
df_sample_lang_article.to_excel('df_sample_lang_article_translation.xlsx')

In [ ]:
#df_sample_lang_article[0:5000].to_excel('df_sample_lang_article_translation_file1.xlsx')
df_sample_lang_article[5000:10000].to_excel('df_sample_lang_article_translation_file2.xlsx')
df_sample_lang_article[10000:15000].to_excel('df_sample_lang_article_translation_file3.xlsx')
df_sample_lang_article[15000:len(df_sample_lang_article)].to_excel('df_sample_lang_article_translation_file4.xlsx')

In [ ]:
# translated version; files have been moved or deleted
temp1 = pd.read_excel('translation/df_sample_lang_article_translation_file1.xlsx')
temp2 = pd.read_excel('translation/df_sample_lang_article_translation_file2.xlsx')
temp3 = pd.read_excel('translation/df_sample_lang_article_translation_file3.xlsx')
temp4 = pd.read_excel('translation/df_sample_lang_article_translation_file4.xlsx')


In [ ]:
df_translation = pd.concat([temp1, temp2, temp3, temp4]) 

In [ ]:
# merging those articles in other languages with those in English
import numpy as np

df_translation['article_text2'] = np.where(df_translation['article_text_translated'].isnull(), 
                                           df_translation['article_text'],
                                           df_translation['article_text_translated'])


In [ ]:
df_translation.columns

In [ ]:
test = df_translation.groupby(['source','LANGUAGE'])['article_text_translated'].count().reset_index()


In [ ]:
test[test['LANGUAGE']=='es']['article_text_translated'].sum()

In [ ]:
test[test['LANGUAGE']=='fil']['article_text_translated'].sum()

In [ ]:
test[test['LANGUAGE']=='ko']['article_text_translated'].sum()

In [ ]:
len(test[test['LANGUAGE']=="es"])

In [ ]:
df_sample_lang_list = list(set(df_sample_lang.source.values.tolist()))
df_source_list = list(set(df_nna.source.values.tolist()))


In [ ]:
df_source_list_en = [i for i in df_source_list if i not in df_sample_lang_list]

In [ ]:
print(len(df_source_list_en))
print(len(df_source_list))
print(len(df_sample_lang_list))

In [ ]:
df_source_list_en_only = pd.DataFrame(df_source_list_en, columns = ["source"])

In [ ]:
df_source_list_en_only['en_source'] = 1

In [ ]:
df_source_list_en_only = pd.merge(df_source_list_en_only, df_nna, how="left", on="source")

In [ ]:
df_translation_done = df_translation[['source', 'website', 'article_url', 'article_title',
       'article_author', 'date_published', 'article_text2']]


In [ ]:
df_source_merge = pd.concat([df_source_list_en_only, df_translation_done])

In [ ]:
print(len(df_nna))
print(len(df_source_merge))

In [ ]:
df_source_merge['article_final'] = np.where(df_source_merge['article_text2'].isnull(), df_source_merge['article_text'], df_source_merge['article_text2']) #CO: consider double-checking. I don't see anything wrong but I'm not familiar with np.where and this step seems important.)

In [ ]:
df_source_merge['article_final'].isnull().sum()

In [ ]:
df_source_merge_final = df_source_merge.drop(columns = ['article_text', 'article_text2'])

In [ ]:
df_source_merge_final.to_pickle('Georgia_data_v2_translation.pkl') # this is a first version of merge data with translation (deleted)

## Prep for topic analysis

In [ ]:
%cd /content/drive/MyDrive/Georgia/Topic Dictionaries

In [ ]:
# lemmatization if needed
import spacy
import re

nlp = spacy.load("en_core_web_sm", disable=['ner', 'parser']) # disabling Named Entity Recognition for speed


In [ ]:
def get_lemmas_num(text): 

    regex = re.compile('[^a-zA-Z0-9]')

    lemmas = []
    
    doc = nlp(text)

    for token in doc: 
        if ((token.is_stop == False) and (token.is_punct == False)) and (token.pos_ != 'PRON') and (token.text != 'NA'): # spaCy has some issues by printing pronoun tagger
            lemmas.append(regex.sub('', token.lemma_)) #CO: consider double-checking
    
    return ' '.join(lemmas)

In [ ]:
df_source_merge_final['article_lemma_num'] = df_source_merge_final['article_final'].str.lower().astype(str).apply(get_lemmas_num) # lematization not require capitalization 


In [ ]:
#df_source_merge_final.to_pickle('Georgia_data_v2_translation_ner_lem.pkl') # commented this line to prevent from saving it

## Prep for place analysis

In [ ]:
%cd /content/drive/MyDrive/Georgia/Locations Analysis

In [ ]:
import pandas as pd #CO: alread imported above

df_source_merge_final = pd.read_pickle('/content/drive/MyDrive/Georgia/Web Scraping/Georgia_data_v2_translation_ner2_lem.pkl') #CO: file name differs from what I'm seeing in the folder


In [ ]:
# lemmatization if needed
import spacy #CO: alread imported above
from spacy import displacy

sp = spacy.load('en_core_web_sm', disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])


In [ ]:
#CO: this also seems to be in the script "(2)_Georgia_place_analysis.ipynb". Maybe you only need it once?

def text_ner_place(text):
  text = sp(text)
  
  ner_list = []
  
  for entity in text.ents:
    if entity.label_ == "FAC":
      ner_list.append(entity.text + " (" + entity.label_ + ")")
    if entity.label_ == "GPE":
      ner_list.append(entity.text + " (" + entity.label_ + ")")
    if entity.label_ == "LOC":
      ner_list.append(entity.text + " (" + entity.label_ + ")")
    else:
      None
  return str(set(ner_list))


In [ ]:
df_source_merge_final['ner_places'] = df_source_merge_final['article_final'].astype(str).apply(text_ner_place)

In [ ]:
#df_source_merge_final.to_pickle('Georgia_data_v2_translation_ner.pkl') # commented this line to prevent from saving it